# Implementation of HiFi-GAN

- 이번에도 GitHub로부터 구조를 파악한 후에 직접 코드를 구현한다.
- 참고할 GitHub는 HiFi-GAN 논문 저자의 GitHub이다: `https://github.com/jik876/hifi-gan`

**[확인]**
- 하드 코딩이 되어 있으므로, 이를 보완해야 한다.
- 그러나 다음과 같은 이유로 하드 코딩을 하였다.
    1. Jupyter Notebook으로 작성하였다. 즉, hyperparameters를 따로 분리해야 할 이유가 없다.
    2. 하드 코딩이 hyperparameters를 파악하는 데에 걸리는 시간을 대폭 줄여준다.
    3. 이미 구현된 모델이므로 모델의 hyperparameters 변동성이 매우 적다.

# 0. Import Libraries & Hyperparameters

## 0.1. Import Libraries

In [1]:
# 1. Data Preprocessing
from jamo import hangul_to_jamo
import pandas as pd
import numpy as np
import os, glob, librosa, re, scipy
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split
import torch

# 2.1. Generator
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm, spectral_norm
import time

# 3. Training
import matplotlib.pyplot as plt
import itertools
from torch.utils.tensorboard import SummaryWriter

## 0.2. Hyperparameters

In [2]:
# Hyperparameters for Generating Mel-spectrogram 
sample_rate = 22050
preemphasis = 0.97
n_fft = 1024
hop_length = 256
win_length = 1024
ref_db = 20
max_db = 100
mel_dim = 80

# Major Hyperparameters
batch_size = 8
logging_step = 10
validation_step = 100
checkpoint_step = 1000

# 1. Data Preprocessing

- Tacotron2의 Dataset과 Dataloader를 그대로 가져온다.
- 데이터는 HiFi-GAN 구현에 맞게 재구성한다.

## 1.1. Text Preprocessing

In [34]:
PAD = '_'
EOS = '~'
SPACE = ' '

JAMO_LEADS = "".join([chr(_) for _ in range(0x1100, 0x1113)])
JAMO_VOWELS = "".join([chr(_) for _ in range(0x1161, 0x1176)])
JAMO_TAILS = "".join([chr(_) for _ in range(0x11A8, 0x11C3)])

VALID_CHARS = JAMO_LEADS + JAMO_VOWELS + JAMO_TAILS + SPACE
symbols = PAD + EOS + VALID_CHARS

_symbol_to_id = {s: i for i, s in enumerate(symbols)}
_id_to_symbol = {i: s for i, s in enumerate(symbols)}

# text를 초성, 중성, 종성으로 분리하여 id로 반환하는 함수
def text_to_sequence(text):
    sequence = []
    if not 0x1100 <= ord(text[0]) <= 0x1113:
        text = ''.join(list(hangul_to_jamo(text)))
    for s in text:
        sequence.append(_symbol_to_id[s])
    sequence.append(_symbol_to_id['~'])
    return sequence

def sequence_to_text(sequence):
    result = ''
    for symbol_id in sequence:
        if symbol_id in _id_to_symbol:
            s = _id_to_symbol[symbol_id]
            result += s
    return result.replace('}{', ' ')

In [35]:
data_dir = "C:/Users/Poco/Jupyter/PaperReview/dataset/kss"
text_dir = data_dir + "/transcript.v.1.4.txt"
filters = '([.,!?])'

metadata = pd.read_csv(text_dir, dtype='object', sep='|', header=None)
text = metadata[3].values

out_dir = data_dir + '/data'
os.makedirs(out_dir, exist_ok=True)
os.makedirs(out_dir + '/text', exist_ok=True)
os.makedirs(out_dir + '/wav', exist_ok=True)
os.makedirs(out_dir + '/mel', exist_ok=True)

# text
print('Load Text')
text_len = []
for idx, s in enumerate(tqdm(text)):
    # 문자열에서 정규표현식을 이용하여 특정 문자열을 필터링하고,
    # 이를 빈 문자열('')로 대체한다.
    sentence = re.sub(re.compile(filters), '', s)
    sentence = text_to_sequence(sentence)
    
    text_len.append(len(sentence))
    text_name = 'kss-text-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/text', text_name), sentence, allow_pickle=False)
print('Text Done')

Load Text


100%|██████████████████████████████████████████████████████████████████████████| 12854/12854 [00:04<00:00, 2686.51it/s]

Text Done


## 1.2. Audio Preprocessing

In [44]:
# HiFi-GAN 저자 구현
def mel_spectrogram(y, n_fft=1024, num_mels=80, sampling_rate=22050, hop_size=256, win_size=1024, fmin=0, fmax=8000, center=False):
    """
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))
    """

    mel = librosa.filters.mel(sampling_rate, n_fft, num_mels, fmin, fmax)
    
    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=torch.hann_window(win_size).to(y.device),
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(torch.from_numpy(mel).float().to(y.device), spec)
    spec = torch.log(torch.clamp(spec, min=1e-5) * 1)

    return spec

In [45]:
# audio
wav_dir = metadata[0].values

print('Load Audio')
for idx, fn in enumerate(tqdm(wav_dir)):
    file_dir = data_dir + '/' + fn
    wav, _ = librosa.load(file_dir, sr=sample_rate)
    wav, _ = librosa.effects.trim(wav) # 묵음 제거
    
    mel = mel_spectrogram(torch.tensor(wav).unsqueeze(0)) # (1, 80, F)
    mel = mel.squeeze(0).T # (F, 80)
    mel = mel.numpy()

    mel_name = 'kss-mel-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/mel', mel_name), mel, allow_pickle=False)
    
    # Wavenet vocoder에서 이용할 wav 파일을 추가로 저장함.
    wav_name = 'kss-wav-%05d.npy' % idx
    np.save(os.path.join(out_dir + '/wav', wav_name), wav, allow_pickle=False)

print('Audio Done')

Load Audio


100%|████████████████████████████████████████████████████████████████████████████| 12854/12854 [13:52<00:00, 15.43it/s]

Audio Done


## 1.3. Dataset and DataLoader

In [3]:
data_dir = "C:/Users/Poco/Jupyter/PaperReview/dataset/kss/data"

class TextMelDataset(Dataset):
    def __init__(self, data_dir):
        self.text_list = sorted(glob.glob(os.path.join(data_dir + '/text', '*.npy')))
        self.mel_list = sorted(glob.glob(os.path.join(data_dir + '/mel', '*.npy')))
        self.wav_list = sorted(glob.glob(os.path.join(data_dir + '/wav', '*.npy')))
        
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, idx):
        text = torch.from_numpy(np.load(self.text_list[idx]))
        text_len = len(text)
        
        mel = torch.from_numpy(np.load(self.mel_list[idx]))
        mel_len = mel.shape[0]
        
        wav = torch.from_numpy(np.load(self.wav_list[idx]))
        wav_len = wav.shape[0]
        return (text, text_len, mel, mel_len, wav, wav_len)

def collate_fn(batch):
    text = []
    text_len = []
    mel = []
    mel_len = []
    wav = []
    wav_len = []
    
    for t, tl, m, ml, w, wl in batch:
        text.append(t)
        text_len.append(tl)
        mel.append(m)
        mel_len.append(ml)
        wav.append(w)
        wav_len.append(wl)
        
    max_text_len = max(text_len)
    max_mel_len = max(mel_len)
    max_wav_len = max(wav_len)
    
    # text zero_padding
    padded_text_batch = torch.zeros((len(batch), max_text_len), dtype=torch.int32)
    for i, x in enumerate(text):
        padded_text_batch[i, :len(x)] = torch.Tensor(x)
    
    # mel zero_padding
    padded_mel_batch = torch.zeros((len(batch), max_mel_len, mel_dim), dtype=torch.float32)
    for i, x in enumerate(mel):
        padded_mel_batch[i, :x.shape[0], :x.shape[1]] = torch.Tensor(x)
        
    # wav zero_padding
    padded_wav_batch = torch.zeros((len(batch), max_wav_len), dtype=torch.float32)
    for i, x in enumerate(wav):
        padded_wav_batch[i, :x.shape[0]] = torch.Tensor(x)
        
    return padded_text_batch, text_len, padded_mel_batch, mel_len, padded_wav_batch, wav_len

dataset = TextMelDataset(data_dir)

# Split dataset into training and validation
train_ratio = 0.99
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, shuffle=True,
                              batch_size=batch_size, collate_fn=collate_fn,
                              pin_memory=True)
val_dataloader = DataLoader(val_dataset,  shuffle=False,
                            batch_size=1, collate_fn=collate_fn,
                            pin_memory=True)

In [4]:
# DataLoader 객체를 반복자로 변환
dataiter = iter(train_dataloader)

# 데이터 한 번 추출
batch = next(dataiter)

print('batch[0](text) shape(B, Max_T):', batch[0].shape) # Max_T: The number of text length
print('batch[1](text_len) len(B):', len(batch[1]))
print('batch[2](mel) shape(B, Max_F, mel_dim):', batch[2].shape) # Max_F: The number of Mel-spectrogram frames
print('batch[3](mel_len) len(B):', len(batch[3]))
print('batch[4](wav) shape(B, Max_L):', batch[4].shape) # Max_L: length of time(seconds) * sampling_rate(22050)
print('batch[5](wav_len) len(B):', len(batch[5]))
print('Total:', dataset.__len__())
print('num_of_batches:', len(train_dataloader))

batch[0](text) shape(B, Max_T): torch.Size([8, 52])
batch[1](text_len) len(B): 8
batch[2](mel) shape(B, Max_F, mel_dim): torch.Size([8, 374, 80])
batch[3](mel_len) len(B): 8
batch[4](wav) shape(B, Max_L): torch.Size([8, 95744])
batch[5](wav_len) len(B): 8
Total: 12854
num_of_batches: 1591


In [5]:
# DataLoader 객체를 반복자로 변환
dataiter = iter(val_dataloader)

# 데이터 한 번 추출
batch = next(dataiter)

print('batch[0](text) shape(B, Max_T):', batch[0].shape) # Max_T: The number of text length
print('batch[1](text_len) len(B):', len(batch[1]))
print('batch[2](mel) shape(B, Max_F, mel_dim):', batch[2].shape) # Max_F: The number of Mel-spectrogram frames
print('batch[3](mel_len) len(B):', len(batch[3]))
print('batch[4](wav) shape(B, Max_L):', batch[4].shape) # Max_L: length of time(seconds) * sampling_rate(22050)
print('batch[5](wav_len) len(B):', len(batch[5]))
print('Total:', dataset.__len__())
print('num_of_batches:', len(val_dataloader))

batch[0](text) shape(B, Max_T): torch.Size([1, 47])
batch[1](text_len) len(B): 1
batch[2](mel) shape(B, Max_F, mel_dim): torch.Size([1, 292, 80])
batch[3](mel_len) len(B): 1
batch[4](wav) shape(B, Max_L): torch.Size([1, 74752])
batch[5](wav_len) len(B): 1
Total: 12854
num_of_batches: 129


# 2. Model

## 2.1. Generator

In [6]:
# V2 model을 기준으로 한다.
class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size):
        """
        channels: 
        kernel_size: 3, 7, 11 중 하나
        """
        super(ResBlock, self).__init__()
        # padding = (kernel_size-1)*dilation//2 ("same")
        self.convs1 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.convs2 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=3,
                                  padding=(kernel_size-1)*3//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.convs3 = nn.ModuleList([
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=5,
                                  padding=(kernel_size-1)*5//2)),
            weight_norm(nn.Conv1d(channels, channels, kernel_size, stride=1, dilation=1,
                                  padding=(kernel_size-1)*1//2))
        ])
        self.modules = [self.convs1, self.convs2, self.convs3]
        
        # 평균이 0, 표준편차가 0.01인 정규분포로 가중치 초기화
        for module in self.modules:
            for conv in module:
                nn.init.normal_(conv.weight, mean=0.0, std=0.01)
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, channels, F) # mel-spectrogram으로부터 얻어진 input features
        =====outputs=====
        x: (B, channels, F) # mel-spectrogram으로부터 얻어진 output features
        """
        for module in self.modules:
            for conv in module:
                y = F.leaky_relu(x, 0.1)
                y = conv(y)
            x = x + y
        return x

class MRF(nn.Module):
    def __init__(self, channels):
        """
        channels: 
        """
        super(MRF, self).__init__()
        self.res_blocks = nn.ModuleList([
            ResBlock(channels, kernel_size=3),
            ResBlock(channels, kernel_size=7),
            ResBlock(channels, kernel_size=11),
        ])
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, channels, F)
        =====outputs=====
        x: (B, channels, F)
        """
        skip_list = []
        for res_block in self.res_blocks:
            skip_x = res_block(x)
            skip_list.append(skip_x)
        x = sum(skip_list) / len(self.res_blocks)
        return x
    
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.pre_conv = weight_norm(nn.Conv1d(80, 128, kernel_size=7, stride=1, dilation=1,
                                              padding=(7-1)//2)) # (B, 80, F) -> (B, 128, F)
        nn.init.normal_(self.pre_conv.weight, mean=0.0, std=0.01) # 논문 저자 구현에는 없음.
        
        self.up_convs = nn.ModuleList()
        self.mrfs = nn.ModuleList()
        ku = [16, 16, 4, 4]
        for i in range(4):
            # ku//2 배 upsampling
            channels = 128//(2**(i+1))
            up_conv = weight_norm(nn.ConvTranspose1d(128//(2**i), channels, kernel_size=ku[i], stride=ku[i]//2,
                                                     padding=(ku[i]-ku[i]//2)//2))
                # (B, 128, F) -(1)-> (B, 64, F*8) -(2)-> (B, 32, F*8*8) -(3)-> (B, 16, F*8*8*2) -(4)-> (B, 8, F*8*8*2*2)
            nn.init.normal_(up_conv.weight, mean=0.0, std=0.01)
            self.up_convs.append(up_conv)
            
            # MRF
            mrf = MRF(channels) # (B, channels, F) -> (B, channels, F)
            self.mrfs.append(mrf)
            
        self.post_conv = weight_norm(nn.Conv1d(8, 1, kernel_size=7, stride=1, dilation=1,
                                               padding=(7-1)//2)) # (B, 8, F*256) -> (B, 1, F*256)
        nn.init.normal_(self.post_conv.weight, mean=0.0, std=0.01)
        
    def forward(self, x):
        """
        =====inputs=====
        x: (B, 80, F) # mel_spectrogram
        =====outputs=====
        x: (B, 1, F*256) # waveform
        """
        x = self.pre_conv(x) # (B, 80, F) -> (B, 128, F)
        for i in range(4):
            x = F.leaky_relu(x, 0.1)
            x = self.up_convs[i](x)
            x = self.mrfs[i](x)
            # final: (B, 128, F) -> (B, 8, F*256)
        x = F.leaky_relu(x, 0.1)
        x = self.post_conv(x) # (B, 8, F*256) -> (B, 1, F*256)
        x = torch.tanh(x)
        return x

## 2.2. Discriminator

In [7]:
class SubPD(nn.Module):
    def __init__(self, period):
        """
        period: 2, 3, 5, 7, 11 중 하나
        """
        super(SubPD, self).__init__()
        self.period = period
        
        self.convs = nn.ModuleList()
        channels = 1
        for i in range(1, 5): # 논문 저자의 변형 구현 대신 논문대로 구현함.
            conv = weight_norm(nn.Conv2d(channels, 2**(5+i), kernel_size=(5, 1), stride=(3, 1), dilation=1, padding=0))
            self.convs.append(conv)
            channels = 2**(5+i)
            # (B, 1, [T/p]+1, p) -(1)-> (B, 64, ?, p) -(2)-> (B, 128, ?, p) -(3)-> (B, 256, ?, p) -(4)-> (B, 512, ?, p)
        last_conv = weight_norm(nn.Conv2d(channels, 1024, kernel_size=(5, 1), stride=(1, 1), dilation=1,
                                          padding=(2, 0))) # (B, 512, ?, p) -> (B, 1024, ?, p)
        self.convs.append(last_conv)
        
        self.post_conv = weight_norm(nn.Conv2d(1024, 1, kernel_size=(3, 1), stride=(1, 1), dilation=1,
                                               padding=(1, 0))) # (B, 1024, ?, p) -> (B, 1, ?, p)
        
    def forward(self, waveform):
        """
        =====inputs=====
        waveform: (B, 1, T)
        =====outputs=====
        x: (B, ?) # flatten된 real/fake 벡터 (0~1(?))
        features: feature를 모두 모아놓은 list (Feature Matching Loss를 계산하기 위함.)
        """
        features = []
        
        B, _, T = waveform.size()
        P = self.period
        # padding
        if T % P != 0:
            padding = P - (T % P)
            waveform = F.pad(waveform, (0, padding), "reflect") # 앞쪽에 0, 뒤쪽에 padding만큼 패딩, reflect는 마치 거울에 반사되듯이 패딩함.
                # ex) [1, 2, 3, 4, 5]를 앞쪽에 2, 뒤쪽에 3만큼 reflect 모드로 padding -> [3, 2, 1, 2, 3, 4, 5, 4, 3, 2]
            T += padding
        # reshape
        x = waveform.view(B, 1, T//P, P) # (B, 1, [T/P]+1, P)
        
        for conv in self.convs:
            x = conv(x)
            x = F.leaky_relu(x, 0.1)
            features.append(x)
        x = self.post_conv(x)
        features.append(x)
        x = torch.flatten(x, 1, -1) # index 1번째 차원부터 마지막 차원까지 flatten | (B, ?)
        ##### sigmoid 함수나 cliping 과정을 거치지 않아도 되는가...?
        return x, features

class MPD(nn.Module):
    def __init__(self):
        super(MPD, self).__init__()
        self.sub_pds = nn.ModuleList([
            SubPD(2), SubPD(3), SubPD(5), SubPD(7), SubPD(11), 
        ]) # (B, 1, T) -> (B, ?), features list
        
    def forward(self, real_waveform, gen_waveform):
        """
        =====inputs=====
        real_waveform: (B, 1, T) # 실제 음성
        gen_waveform: (B, 1, T) # 생성 음성
        =====outputs=====
        real_outputs: (B, ?) list (len=5) # 실제 음성에 대한 SubPD outputs list
        gen_outputs: (B, ?) list # 생성 음성에 대한 SubPD outputs list
        real_features: features list # 실제 음성에 대한 SubPD features list
        gen_features: features list # 생성 음성에 대한 SubPD features list
        """
        real_outputs, gen_outputs, real_features, gen_features = [], [], [], []
        for sub_pd in self.sub_pds:
            real_output, real_feature = sub_pd(real_waveform)
            gen_output, gen_feature = sub_pd(gen_waveform)
            real_outputs.append(real_output)
            gen_outputs.append(gen_output)
            real_features.append(real_feature)
            gen_features.append(gen_feature)
        return real_outputs, gen_outputs, real_features, gen_features
    
class SubSD(nn.Module):
    def __init__(self, first=False):
        """
        first: boolean (first가 True이면 spectral normalization을 적용한다.)
        """
        super(SubSD, self).__init__()
        norm = spectral_norm if first else weight_norm # first가 True이면 spectral_norm, 그렇지 않으면 weight_norm
        self.convs = nn.ModuleList([ # Mel-GAN 논문에 맞게 구현
            norm(nn.Conv1d(1, 16, kernel_size=15, stride=1, padding=(15-1)//2)), # (B, 1, T) -> (B, 16, T)
            norm(nn.Conv1d(16, 64, kernel_size=41, stride=4, groups=4, padding=(41-1)//2)), # (B, 16, T) -> (B, 64, T/4(?))
            norm(nn.Conv1d(64, 256, kernel_size=41, stride=4, groups=16, padding=(41-1)//2)), # (B, 64, T/4(?)) -> (B, 256, T/16(?))
            norm(nn.Conv1d(256, 1024, kernel_size=41, stride=4, groups=64, padding=(41-1)//2)), # (B, 256, T/16(?)) -> (B, 1024, T/64(?))
            norm(nn.Conv1d(1024, 1024, kernel_size=41, stride=4, groups=256, padding=(41-1)//2)), # (B, 1024, T/64(?)) -> (B, 1024, T/256(?))
            norm(nn.Conv1d(1024, 1024, kernel_size=5, stride=1, padding=(5-1)//2)) # (B, 1024, T/256(?)) -> (B, 1024, T/256(?))
        ])
        self.post_conv = norm(nn.Conv1d(1024, 1, kernel_size=3, stride=1, padding=(3-1)//2)) # (B, 1024, ?) -> (B, 1, ?)
        
    def forward(self, waveform):
        """
        =====inputs=====
        waveform: (B, 1, T)
        =====outputs=====
        x: (B, ?) # flatten된 real/fake 벡터 (0~1(?))
        features: feature를 모두 모아놓은 list (Feature Matching Loss를 계산하기 위함.)
        """
        features = []
        x = waveform
        for conv in self.convs:
            x = conv(x)
            x = F.leaky_relu(x, 0.1)
            features.append(x)
        x = self.post_conv(x) # (B, 1, ?)
        features.append(x)
        x = x.squeeze(1) # (B, ?)
        ##### sigmoid 함수나 cliping 과정을 거치지 않아도 되는가...?
        return x, features
    
class MSD(nn.Module):
    def __init__(self):
        super(MSD, self).__init__()
        self.sub_sds = nn.ModuleList([
            SubSD(first=True), SubSD(), SubSD() 
        ]) # (B, 1, T) -> (B, ?), features list
        self.avgpool = nn.AvgPool1d(kernel_size=4, stride=2, padding=2) # x2 down sampling
        
    def forward(self, real_waveform, gen_waveform):
        """
        =====inputs=====
        real_waveform: (B, 1, T) # 실제 음성
        gen_waveform: (B, 1, T) # 생성 음성
        =====outputs=====
        real_outputs: (B, ?) list (len=3) # 실제 음성에 대한 SubSD outputs list
        gen_outputs: (B, ?) list # 생성 음성에 대한 SubSD outputs list
        real_features: features list # 실제 음성에 대한 SubSD features list
        gen_features: features list # 생성 음성에 대한 SubSD features list
        """
        real_outputs, gen_outputs, real_features, gen_features = [], [], [], []
        for idx, sub_sd in enumerate(self.sub_sds):
            if idx != 0:
                real_waveform = self.avgpool(real_waveform)
                gen_waveform = self.avgpool(gen_waveform)
            real_output, real_feature = sub_sd(real_waveform)
            gen_output, gen_feature = sub_sd(gen_waveform)
            real_outputs.append(real_output)
            gen_outputs.append(gen_output)
            real_features.append(real_feature)
            gen_features.append(gen_feature)
        return real_outputs, gen_outputs, real_features, gen_features

### Debugging

In [7]:
device = torch.device("cpu")

# Generator in cpu
start = time.time()

# mel = torch.randn(64, 80, 100).to(device)
mel = batch[2].transpose(1, 2).to(device) # (64, 80, ?)
Gen = Generator().to(device)
wav = Gen(mel)
print(wav.size())

end = time.time()
print(f"{device}: {end-start}s")

torch.Size([64, 1, 120064])
cpu: 13.261306047439575s


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Generator in cuda
start = time.time()

# mel = torch.randn(64, 80, 100).to(device)
mel = batch[2].transpose(1, 2).to(device) # (64, 80, ?)
Gen = Generator().to(device)
wav = Gen(mel)
print(wav.size())

end = time.time()
print(f"{device}: {end-start}s")

torch.Size([16, 1, 105728])
cuda: 1.6936736106872559s


In [19]:
device = torch.device("cpu")

# Discriminator in cpu
start = time.time()

# real_wav = torch.randn(64, 1, 25600).to(device)
# gen_wav = torch.randn(64, 1, 25600).to(device)
real_wav = batch[4].unsqueeze(1).to(device)
gen_wav = wav.to(device)
mpd = MPD().to(device)
msd = MSD().to(device)
ro, go, rf, gf = mpd(real_wav, gen_wav)
ro2, go2, rf2, gf2 = msd(real_wav, gen_wav)
print(ro[0].size(), go[0].size(), len(rf), len(gf[0]))
print(ro2[0].size(), go2[0].size(), len(rf2), len(gf2[0]))

end = time.time()
print(f"{device}: {end-start}s")

torch.Size([64, 1438]) torch.Size([64, 1442]) 5 6
torch.Size([64, 456]) torch.Size([64, 457]) 3 7
cpu: 27.11948275566101s


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Discriminator in cuda
start = time.time()

# real_wav = torch.randn(64, 1, 25600).to(device)
# gen_wav = torch.randn(64, 1, 25600).to(device)
real_wav = batch[4].unsqueeze(1).to(device)
gen_wav = wav.to(device)
mpd = MPD().to(device)
msd = MSD().to(device)
ro, go, rf, gf = mpd(real_wav, gen_wav)
ro2, go2, rf2, gf2 = msd(real_wav, gen_wav)
print(ro[0].size(), go[0].size(), len(rf), len(gf[0]))
print(ro2[0].size(), go2[0].size(), len(rf2), len(gf2[0]))

end = time.time()
print(f"{device}: {end-start}s")

torch.Size([16, 1300]) torch.Size([16, 1302]) 5 6
torch.Size([16, 413]) torch.Size([16, 413]) 3 7
cuda: 0.3021814823150635s


In [9]:
model = Generator()
sum(p.numel() for p in model.parameters() if p.requires_grad)

928514

# 3. Training

## 3.0. Plot mel-spectrogram

In [8]:
def plot_mel(mel, step=None, mel_loss=None):
    """
    =====inputs=====
    mel: (80, F) # 입력 시 .detach().cpu().numpy() 를 적용하는 것을 잊지 말 것!
    """
    fig, ax = plt.subplots(figsize=(10, 4))
    im = ax.imshow(mel, origin="lower", aspect="auto")
    plt.title(f"Step: {step} | Mel_Loss: {mel_loss}")
    plt.xlabel("time")
    plt.ylabel("frequency_bin")
    fig.colorbar(im, ax=ax)
    
    fig.canvas.draw()
    plt.close()
    
    return fig

## 3.1. Loss Functions

In [9]:
def GAN_Loss_Generator(gen_outputs):
    """
    gen_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    """
    loss = 0
    for DG in gen_outputs:
        loss += torch.mean((DG-1)**2)
    return loss

def GAN_Loss_Discriminator(real_outputs, gen_outputs):
    """
    real_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    gen_outputs: (B, ?) list # MPD(len=5) 또는 MSD(len=3)의 출력
    """
    loss = 0
    for D, DG in zip(real_outputs, gen_outputs):
        loss += torch.mean((D-1)**2 + DG**2)
    return loss

def Mel_Spectrogram_Loss(real_mel, gen_mel):
    """
    real_mel: (B, F, 80) # Dataloader로부터 가져온 mel-spectrogram
    gen_mel: (B, F, 80) # Generator가 생성한 waveform의 mel-spectrogram
    """
    loss = F.l1_loss(real_mel, gen_mel)
    return 45*loss

def Feature_Matching_Loss(real_features, gen_features):
    """
    real_features: (?, ..., ?) list of list # MPD(len=[5, 6]) 또는 MSD(len=[3, 7])의 출력
    gen_features: (?, ..., ?) list of list # MPD(len=[5, 6]) 또는 MSD(len=[3, 7])의 출력
    """
    loss = 0
    for Ds, DGs in zip(real_features, gen_features):
        for D, DG in zip(Ds, DGs):
            loss += torch.mean(torch.abs(D - DG))
    return 2*loss

def Final_Loss_Generator(mpd_gen_outputs, mpd_real_features, mpd_gen_features,
                         msd_gen_outputs, msd_real_features, msd_gen_features,
                         real_mel, gen_mel):
    """
    =====inputs=====
    [:3]: MPD outputs 뒤쪽 3개
    [3:6]: MSD outputs 뒤쪽 3개
    [7:8]: real_mel and gen_mel
    =====outputs=====
    Generator_Loss
    Mel_Loss
    """
    Gen_Adv1 = GAN_Loss_Generator(mpd_gen_outputs)
    Gen_Adv2 = GAN_Loss_Generator(msd_gen_outputs)
    Adv = Gen_Adv1 + Gen_Adv2
    FM1 = Feature_Matching_Loss(mpd_real_features, mpd_gen_features)
    FM2 = Feature_Matching_Loss(msd_real_features, msd_gen_features)
    FM = FM1 + FM2
    Mel_Loss = Mel_Spectrogram_Loss(real_mel, gen_mel)
    Generator_Loss = Adv + FM + Mel_Loss
    
    return Generator_Loss, Mel_Loss

def Final_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs,
                             msd_real_outputs, msd_gen_outputs):
    """
    =====inputs=====
    [:2]: MPD outputs 앞쪽 2개
    [2:4]: MSD outputs 앞쪽 2개
    =====outputs=====
    Discriminator_Loss
    """
    Disc_Adv1 = GAN_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs)
    Disc_Adv2 = GAN_Loss_Discriminator(msd_real_outputs, msd_gen_outputs)
    Discriminator_Loss = Disc_Adv1 + Disc_Adv2
    
    return Discriminator_Loss

## 3.2. Mel-spectrogram Converter

In [10]:
# HiFi-GAN 저자 구현
def mel_spectrogram(y, n_fft=1024, num_mels=80, sampling_rate=22050, hop_size=256, win_size=1024, fmin=0, fmax=8000, center=False):
    """
    if torch.min(y) < -1.:
        print('min value is ', torch.min(y))
    if torch.max(y) > 1.:
        print('max value is ', torch.max(y))
    """

    mel = librosa.filters.mel(sampling_rate, n_fft, num_mels, fmin, fmax)
    
    y = torch.nn.functional.pad(y.unsqueeze(1), (int((n_fft-hop_size)/2), int((n_fft-hop_size)/2)), mode='reflect')
    y = y.squeeze(1)

    spec = torch.stft(y, n_fft, hop_length=hop_size, win_length=win_size, window=torch.hann_window(win_size).to(y.device),
                      center=center, pad_mode='reflect', normalized=False, onesided=True)

    spec = torch.sqrt(spec.pow(2).sum(-1)+(1e-9))

    spec = torch.matmul(torch.from_numpy(mel).float().to(y.device), spec)
    spec = torch.log(torch.clamp(spec, min=1e-5) * 1)

    return spec

In [11]:
# Tacotron1 구현 (폐기, 해당 구현에서는 사용하지 않음.)
def wav2mel(batch_wav):
    """
    =====inputs=====
    batch_wav: (B, L) # (B, 1, L)은 반드시 squeeze를 이용해야 한다. .detach().cpu().numpy() 작업도 필요하다!
    =====outputs=====
    batch_mel_spec: (B, 80, F) # torch.tensor로 반환한다
    """
    B, _ = batch_wav.shape
    mel_spec_list = []
    for i in range(B):
        wav = batch_wav[i, :]
        # 아래 필터는 고주파 성분 강조를 통해 음성 인식 성능을 향상시킨다.
        filtered_wav = scipy.signal.lfilter([1, -preemphasis], [1], wav)
        # stft 결과값(복소수) (진폭 정보를 추출)
        stft = librosa.stft(filtered_wav, n_fft=n_fft, hop_length=hop_length, win_length=win_length)
        stft = np.abs(stft)
        mel_filter = librosa.filters.mel(sample_rate, n_fft, mel_dim)
        mel_spec = np.dot(mel_filter, stft) # Mel-spec 생성

        # dB 스케일(dB scale)을 적용 (사람의 청각 특성에 맞게 설계된 척도)
        mel_spec = 20 * np.log10(np.maximum(1e-5, mel_spec))

        # 정규화 및 클리핑
        mel_spec = np.clip((mel_spec - ref_db + max_db) / max_db, 1e-8, 1) # (80, F)
        mel_spec_list.append(mel_spec)
        
    Max_F = max([array.shape[1] for array in mel_spec_list]) # Max_F를 찾아냄
    batch_mel_spec = torch.zeros((B, 80, Max_F))
    for i, mel in enumerate(mel_spec_list):
        batch_mel_spec[i, :mel.shape[0], :mel.shape[1]] = torch.Tensor(mel)
    
    return batch_mel_spec

## 3.3. Main Training Function

In [12]:
def train(model_name, check_step):
    """
    (NEW) check_step = None
    (LOAD) check_step: 불러오고자 하는 모델의 step
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Define Models & Optimizer
    gen_model = Generator().to(device)
    mpd_model = MPD().to(device)
    msd_model = MSD().to(device)
    
    gen_optim = torch.optim.AdamW(gen_model.parameters(),
                                  lr=0.0002, betas=(0.8, 0.99))
    disc_optim = torch.optim.AdamW(itertools.chain(mpd_model.parameters(),
                                                   msd_model.parameters()),
                                   lr=0.0002, betas=(0.8, 0.99))
    # 오류에 따른 수정: initial_lr 지정
    for param_group in gen_optim.param_groups:
        param_group['initial_lr'] = 0.0002
    
    for param_group in disc_optim.param_groups:
        param_group['initial_lr'] = 0.0002
    
    # Load Models & Optimizer
    epoch = 1
    step = 1
    if check_step is not None:
        os.makedirs('ckpt' + model_name, exist_ok=True)
        check_point = './ckpt/' + model_name + "/ckpt-" + str(check_step) + ".pt"
        ckpt = torch.load(check_point)
        gen_model.load_state_dict(ckpt['gen_model'])
        mpd_model.load_state_dict(ckpt['mpd_model'])
        msd_model.load_state_dict(ckpt['msd_model'])
        gen_optim.load_state_dict(ckpt['gen_optim'])
        disc_optim.load_state_dict(ckpt['disc_optim'])
        epoch = ckpt['epoch']
        step = ckpt['step']
        print(f'Load {model_name} | Step {check_step}')
    
    # cuDNN의 벤치마크 모드를 활성화. 합성곱과 풀링 연산 등을 최적화.
    torch.backends.cudnn.benchmark = True
    
    # Define Scheduler
    gen_scheduler = torch.optim.lr_scheduler.ExponentialLR(gen_optim, gamma=0.999,
                                                           last_epoch=epoch)
    disc_scheduler = torch.optim.lr_scheduler.ExponentialLR(disc_optim, gamma=0.999,
                                                            last_epoch=epoch)
    
    # Training Mode Activation
    gen_model.train()
    mpd_model.train()
    msd_model.train()
    
    # tensorboard
    os.makedirs('ckpt' + model_name + '/logs', exist_ok=True)
    sw = SummaryWriter('./ckpt/' + model_name + '/logs')
    
    # Main Training
    """
    mel과 mel_loss를 구분하여 논문 저자 구현을 따라감.
    """
    start = time.time() # start time 초기화
    while True:
        print(f"|| Epoch: {epoch} ||")
        for _, batch in enumerate(train_dataloader):
            _, _, mel, _, wav, _ = batch
            real_mel = mel.to(device) # (B, Max_F, 80)
            real_wav = wav.to(device) # (B, Max_L)
            real_mel = real_mel.transpose(1, 2) # (B, 80, Max_F) # fmax=8000
            real_wav = real_wav.unsqueeze(1) # (B, 1, Max_L)
            _, _, Max_L = real_wav.size()
            
            real_mel_for_loss = mel_spectrogram(real_wav.squeeze(1), fmax=11025)
            
            # Generator
            gen_wav = gen_model(real_mel)
            _, _, Gen_L = gen_wav.size()
            if Max_L < Gen_L:
                gen_wav = gen_wav[:, :, :Max_L]
            else:
                real_wav = real_wav[:, :, :Gen_L]
            
            gen_mel = mel_spectrogram(gen_wav.squeeze(1), fmax=11025)
                """
                Nyquist Frequency에 의하면 sampling rate의 절반에 해당하는 주파수까지 분석이 가능하다.
                현재 sampling rate이 22050이므로, fmax를 그의 절반인 11025로 둔다.
                """
            
            # Discriminator Loss
            
            # MPD
            mpd_real_outputs, mpd_gen_outputs, mpd_real_features, mpd_gen_features = mpd_model(real_wav, gen_wav.detach())
            # MSD
            msd_real_outputs, msd_gen_outputs, msd_real_features, msd_gen_features = msd_model(real_wav, gen_wav.detach())
                # Discriminator에서는 gen_wav에 대한 gradient를 계산하지 않음을 명심하라.

            disc_optim.zero_grad()
            disc_loss = Final_Loss_Discriminator(mpd_real_outputs, mpd_gen_outputs,
                                                 msd_real_outputs, msd_gen_outputs)
            disc_loss.backward()
            disc_optim.step()
            
            # Generator Loss
            
            # MPD
            mpd_real_outputs, mpd_gen_outputs, mpd_real_features, mpd_gen_features = mpd_model(real_wav, gen_wav)
            # MSD
            msd_real_outputs, msd_gen_outputs, msd_real_features, msd_gen_features = msd_model(real_wav, gen_wav)
            
            gen_optim.zero_grad() # 이전 step의 gradient 초기화
            gen_loss, mel_loss = Final_Loss_Generator(mpd_gen_outputs, mpd_real_features, mpd_gen_features,
                                            msd_gen_outputs, msd_real_features, msd_gen_features,
                                            real_mel_for_loss, gen_mel)
            gen_loss.backward() # gradient 계산
            gen_optim.step() # 가중치 업데이트
            
            step += 1
            
            # Logging
            if step % logging_step == 0:
                print(f'| step: {step} | gen_loss: {gen_loss:4.3f} | mel_loss: {mel_loss:4.3f} | {time.time()-start:.3f} sec / {logging_step} steps |')
                # Tensorboard에 기록
                sw.add_scalar("training/gen_loss", gen_loss, step)
                sw.add_scalar("training/mel_loss", mel_loss, step)
                start = time.time() # start time 초기화
            
            # Validation
            if step % validation_step == 0:
                gen_model.eval()
                torch.cuda.empty_cache()
                
                mel_loss_sum = 0
                with torch.no_grad():
                    for i, batch in enumerate(val_dataloader):
                        _, _, mel, _, wav, _ = batch
                        real_mel = mel.to(device) # (1, F, 80)
                        real_wav = wav.to(device) # (1, L)
                        real_mel = real_mel.transpose(1, 2) # (1, 80, F)
                        real_wav = real_wav.unsqueeze(1) # (1, 1, L)
                        _, _, Max_L = real_wav.size()
                        
                        real_mel_for_loss = mel_spectrogram(real_wav.squeeze(1), fmax=11025)
                        
                        # Generator
                        gen_wav = gen_model(real_mel) # (1, L*)
                        _, _, Gen_L = gen_wav.size()
                        if Max_L < Gen_L:
                            gen_wav = gen_wav[:, :, :Max_L]
                        else:
                            real_wav = real_wav[:, :, :Gen_L]
                        
                        gen_mel = mel_spectrogram(gen_wav.squeeze(1), fmax=11025) # (1, 80, F)
                        
                        # Loss
                        mel_loss = Mel_Spectrogram_Loss(real_mel_for_loss, gen_mel)
                        mel_loss_sum += mel_loss
                        
                        if i <= 4: # 그림은 5개 저장
                            sw.add_audio(f'generated/gen_wav_{i}', gen_wav[0], step, sample_rate)
                            sw.add_figure(f'generated/gen_mel_{i})',
                                          plot_mel(gen_mel[0].detach().cpu().numpy(), step, mel_loss),
                                          step)
                        
                    mel_loss_avg = mel_loss_sum / (i+1)
                    sw.add_scalar("validation/mel_loss", mel_loss_avg, step)
                    print(f"Validation mel_loss: {mel_loss_avg}")
                
                gen_model.train()
                start = time.time() # start time 초기화
                        
            # Checkpoint
            if step % checkpoint_step == 0:
                save_dir = './ckpt/' + model_name
                torch.save({
                    'gen_model': gen_model.state_dict(),
                    'mpd_model': mpd_model.state_dict(),
                    'msd_model': msd_model.state_dict(),
                    'gen_optim': gen_optim.state_dict(),
                    'disc_optim': disc_optim.state_dict(),
                    'epoch': epoch,
                    'step': step
                }, os.path.join(save_dir, 'ckpt-{}.pt'.format(step)))
                
                start = time.time() # start time 초기화
                
        # Epoch 증가에 따라 scheduler 조정
        gen_scheduler.step()
        disc_scheduler.step()
        epoch += 1

## 3.4. Main

In [13]:
##### INPUT #####
model_name = "model_01"
check_step = None

In [14]:
if __name__ == "__main__":
    train(model_name, check_step)

|| Epoch: 1 ||


C:\Users\Poco\anaconda3\lib\site-packages\torch\functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\SpectralOps.cpp:804.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


| step: 10 | gen_loss: 76.622 | mel_loss: 71.537 | 17.873 sec / 10 steps |
| step: 20 | gen_loss: 88.308 | mel_loss: 69.358 | 15.704 sec / 10 steps |
| step: 30 | gen_loss: 79.035 | mel_loss: 58.106 | 16.638 sec / 10 steps |
| step: 40 | gen_loss: 79.719 | mel_loss: 55.551 | 20.073 sec / 10 steps |
| step: 50 | gen_loss: 76.628 | mel_loss: 50.745 | 12.550 sec / 10 steps |
| step: 60 | gen_loss: 78.228 | mel_loss: 51.845 | 16.973 sec / 10 steps |
| step: 70 | gen_loss: 84.612 | mel_loss: 58.572 | 10.933 sec / 10 steps |
| step: 80 | gen_loss: 79.729 | mel_loss: 53.447 | 10.142 sec / 10 steps |
| step: 90 | gen_loss: 67.729 | mel_loss: 42.168 | 10.991 sec / 10 steps |
| step: 100 | gen_loss: 79.615 | mel_loss: 54.496 | 13.362 sec / 10 steps |
Validation mel_loss: 62.0623893737793
| step: 110 | gen_loss: 75.573 | mel_loss: 49.532 | 9.300 sec / 10 steps |
| step: 120 | gen_loss: 74.603 | mel_loss: 48.925 | 10.110 sec / 10 steps |
| step: 130 | gen_loss: 67.416 | mel_loss: 42.421 | 10.458 s

| step: 1050 | gen_loss: 37.433 | mel_loss: 25.465 | 5.060 sec / 10 steps |
| step: 1060 | gen_loss: 39.707 | mel_loss: 27.183 | 4.612 sec / 10 steps |
| step: 1070 | gen_loss: 42.315 | mel_loss: 28.568 | 5.958 sec / 10 steps |
| step: 1080 | gen_loss: 39.421 | mel_loss: 25.913 | 6.023 sec / 10 steps |
| step: 1090 | gen_loss: 46.204 | mel_loss: 30.706 | 6.483 sec / 10 steps |
| step: 1100 | gen_loss: 41.925 | mel_loss: 28.411 | 4.130 sec / 10 steps |
Validation mel_loss: 34.76244354248047
| step: 1110 | gen_loss: 36.126 | mel_loss: 23.414 | 5.226 sec / 10 steps |
| step: 1120 | gen_loss: 37.537 | mel_loss: 25.517 | 4.548 sec / 10 steps |
| step: 1130 | gen_loss: 44.807 | mel_loss: 29.754 | 5.554 sec / 10 steps |
| step: 1140 | gen_loss: 42.057 | mel_loss: 27.728 | 4.921 sec / 10 steps |
| step: 1150 | gen_loss: 38.493 | mel_loss: 25.119 | 7.476 sec / 10 steps |
| step: 1160 | gen_loss: 40.536 | mel_loss: 26.280 | 4.542 sec / 10 steps |
| step: 1170 | gen_loss: 44.162 | mel_loss: 29.10

| step: 2080 | gen_loss: 37.683 | mel_loss: 25.116 | 4.522 sec / 10 steps |
| step: 2090 | gen_loss: 41.790 | mel_loss: 27.229 | 4.399 sec / 10 steps |
| step: 2100 | gen_loss: 37.430 | mel_loss: 23.747 | 5.044 sec / 10 steps |
Validation mel_loss: 32.62914276123047
| step: 2110 | gen_loss: 35.748 | mel_loss: 23.553 | 4.496 sec / 10 steps |
| step: 2120 | gen_loss: 35.467 | mel_loss: 23.181 | 5.295 sec / 10 steps |
| step: 2130 | gen_loss: 30.635 | mel_loss: 21.144 | 5.434 sec / 10 steps |
| step: 2140 | gen_loss: 31.956 | mel_loss: 20.205 | 4.861 sec / 10 steps |
| step: 2150 | gen_loss: 38.543 | mel_loss: 25.348 | 4.515 sec / 10 steps |
| step: 2160 | gen_loss: 29.688 | mel_loss: 19.478 | 4.604 sec / 10 steps |
| step: 2170 | gen_loss: 38.271 | mel_loss: 24.641 | 4.347 sec / 10 steps |
| step: 2180 | gen_loss: 37.477 | mel_loss: 23.518 | 4.697 sec / 10 steps |
| step: 2190 | gen_loss: 27.117 | mel_loss: 17.440 | 4.763 sec / 10 steps |
| step: 2200 | gen_loss: 39.412 | mel_loss: 22.90

...

| step: 131980 | gen_loss: 26.314 | mel_loss: 11.609 | 5.221 sec / 10 steps |
| step: 131990 | gen_loss: 27.143 | mel_loss: 10.923 | 5.284 sec / 10 steps |
| step: 132000 | gen_loss: 29.480 | mel_loss: 12.381 | 5.169 sec / 10 steps |
Validation mel_loss: 16.107378005981445
| step: 132010 | gen_loss: 29.682 | mel_loss: 12.340 | 5.553 sec / 10 steps |
| step: 132020 | gen_loss: 25.430 | mel_loss: 10.522 | 5.156 sec / 10 steps |
| step: 132030 | gen_loss: 22.780 | mel_loss: 9.695 | 5.114 sec / 10 steps |
| step: 132040 | gen_loss: 32.834 | mel_loss: 12.284 | 5.301 sec / 10 steps |
| step: 132050 | gen_loss: 29.388 | mel_loss: 13.426 | 5.232 sec / 10 steps |
|| Epoch: 84 ||
| step: 132060 | gen_loss: 26.984 | mel_loss: 11.501 | 9.231 sec / 10 steps |
| step: 132070 | gen_loss: 30.394 | mel_loss: 12.012 | 5.058 sec / 10 steps |
| step: 132080 | gen_loss: 28.482 | mel_loss: 11.337 | 4.894 sec / 10 steps |
| step: 132090 | gen_loss: 30.930 | mel_loss: 12.512 | 4.718 sec / 10 steps |
| step: 1

| step: 132980 | gen_loss: 26.523 | mel_loss: 11.793 | 5.161 sec / 10 steps |
| step: 132990 | gen_loss: 28.327 | mel_loss: 11.280 | 5.077 sec / 10 steps |
| step: 133000 | gen_loss: 29.414 | mel_loss: 10.405 | 5.392 sec / 10 steps |
Validation mel_loss: 16.169164657592773
| step: 133010 | gen_loss: 28.515 | mel_loss: 12.533 | 5.243 sec / 10 steps |
| step: 133020 | gen_loss: 28.369 | mel_loss: 12.316 | 4.918 sec / 10 steps |
| step: 133030 | gen_loss: 32.121 | mel_loss: 12.013 | 4.891 sec / 10 steps |
| step: 133040 | gen_loss: 27.192 | mel_loss: 12.115 | 4.869 sec / 10 steps |
| step: 133050 | gen_loss: 24.592 | mel_loss: 10.574 | 5.053 sec / 10 steps |
| step: 133060 | gen_loss: 25.681 | mel_loss: 10.792 | 4.888 sec / 10 steps |
| step: 133070 | gen_loss: 30.944 | mel_loss: 12.214 | 4.886 sec / 10 steps |
| step: 133080 | gen_loss: 27.219 | mel_loss: 11.836 | 4.825 sec / 10 steps |
| step: 133090 | gen_loss: 23.533 | mel_loss: 10.422 | 5.069 sec / 10 steps |
| step: 133100 | gen_los

| step: 133980 | gen_loss: 30.179 | mel_loss: 13.707 | 5.296 sec / 10 steps |
| step: 133990 | gen_loss: 29.544 | mel_loss: 13.227 | 5.463 sec / 10 steps |
| step: 134000 | gen_loss: 25.980 | mel_loss: 11.451 | 5.582 sec / 10 steps |
Validation mel_loss: 16.114885330200195
| step: 134010 | gen_loss: 30.045 | mel_loss: 10.355 | 5.996 sec / 10 steps |
| step: 134020 | gen_loss: 25.819 | mel_loss: 11.527 | 5.270 sec / 10 steps |
| step: 134030 | gen_loss: 27.094 | mel_loss: 12.465 | 5.408 sec / 10 steps |
| step: 134040 | gen_loss: 23.494 | mel_loss: 10.241 | 5.621 sec / 10 steps |
| step: 134050 | gen_loss: 29.552 | mel_loss: 13.708 | 5.300 sec / 10 steps |
| step: 134060 | gen_loss: 26.176 | mel_loss: 10.727 | 5.192 sec / 10 steps |
| step: 134070 | gen_loss: 30.030 | mel_loss: 12.817 | 5.603 sec / 10 steps |
| step: 134080 | gen_loss: 26.604 | mel_loss: 10.502 | 5.663 sec / 10 steps |
| step: 134090 | gen_loss: 20.824 | mel_loss: 9.289 | 5.961 sec / 10 steps |
| step: 134100 | gen_loss

| step: 134990 | gen_loss: 32.643 | mel_loss: 13.332 | 5.100 sec / 10 steps |
| step: 135000 | gen_loss: 28.043 | mel_loss: 12.386 | 5.962 sec / 10 steps |
Validation mel_loss: 16.17063331604004


KeyboardInterrupt: 